In [1]:
val moviesRdd = sc.textFile("hdfs://localhost:9000/movies/movies.csv")
val ratingRdd = sc.textFile("hdfs://localhost:9000/ratings/ratings.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.80.128:4040
SparkContext available as 'sc' (version = 3.1.3, master = local[*], app id = local-1648402585073)
SparkSession available as 'spark'


moviesRdd: org.apache.spark.rdd.RDD[String] = hdfs://localhost:9000/movies/movies.csv MapPartitionsRDD[1] at textFile at <console>:25
ratingRdd: org.apache.spark.rdd.RDD[String] = hdfs://localhost:9000/ratings/ratings.csv MapPartitionsRDD[3] at textFile at <console>:26


In [2]:
moviesRdd.count()

res0: Long = 9743


In [3]:
ratingRdd.count()

res1: Long = 100837


In [4]:
moviesRdd.first()

res2: String = movieId,title,genres


In [5]:
ratingRdd.first()

res3: String = userId,movieId,rating,timestamp


In [6]:
val header = moviesRdd.first()
val moviesContentRdd = moviesRdd.filter (line => line != header)
moviesContentRdd.take(2)

header: String = movieId,title,genres
moviesContentRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[4] at filter at <console>:27
res4: Array[String] = Array(1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy, 2,Jumanji (1995),Adventure|Children|Fantasy)


In [7]:
val rheader = ratingRdd.first()
val ratingsContentRdd = ratingRdd.filter (line => line != rheader)
ratingsContentRdd.take(2)

rheader: String = userId,movieId,rating,timestamp
ratingsContentRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[5] at filter at <console>:27
res5: Array[String] = Array(1,1,4.0,964982703, 1,3,4.0,964981247)


In [8]:
val moviesParsedRdd = moviesContentRdd
                        .map (line => line.split(","))
moviesParsedRdd.take(4)

moviesParsedRdd: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[6] at map at <console>:27
res6: Array[Array[String]] = Array(Array(1, Toy Story (1995), Adventure|Animation|Children|Comedy|Fantasy), Array(2, Jumanji (1995), Adventure|Children|Fantasy), Array(3, Grumpier Old Men (1995), Comedy|Romance), Array(4, Waiting to Exhale (1995), Comedy|Drama|Romance))


In [9]:
val moviesTupleRdd = moviesParsedRdd.map (arr => (arr(0).toInt, (arr(1), arr(2))))
moviesTupleRdd.take(4)

moviesTupleRdd: org.apache.spark.rdd.RDD[(Int, (String, String))] = MapPartitionsRDD[7] at map at <console>:26
res7: Array[(Int, (String, String))] = Array((1,(Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy)), (2,(Jumanji (1995),Adventure|Children|Fantasy)), (3,(Grumpier Old Men (1995),Comedy|Romance)), (4,(Waiting to Exhale (1995),Comedy|Drama|Romance)))


In [10]:
//case class Movie (id: Int, title: String, genres: String)
//val moviesDataRdd = moviesTupleRdd.map ( t =>Movie(t._1, t._2, t._3))
//moviesDataRdd.take(4)

In [11]:
val ratingParsedRdd = ratingsContentRdd
                        .map (line => line.trim())
                        .filter (line => !line.isEmpty())
                        .map (line => line.split(","))
ratingParsedRdd.take(4)

ratingParsedRdd: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[10] at map at <console>:29
res9: Array[Array[String]] = Array(Array(1, 1, 4.0, 964982703), Array(1, 3, 4.0, 964981247), Array(1, 6, 4.0, 964982224), Array(1, 47, 5.0, 964983815))


In [12]:
val ratingsTupleRdd = ratingParsedRdd.map (rating => (rating(0).toInt, 
                                                      (rating(1).toInt,
                                                      rating(2).toDouble,
                                                      rating(3).toLong)
                                                      ))
ratingsTupleRdd.take(4)                               

ratingsTupleRdd: org.apache.spark.rdd.RDD[(Int, (Int, Double, Long))] = MapPartitionsRDD[11] at map at <console>:26
res10: Array[(Int, (Int, Double, Long))] = Array((1,(1,4.0,964982703)), (1,(3,4.0,964981247)), (1,(6,4.0,964982224)), (1,(47,5.0,964983815)))


In [13]:
//pair Rdd needs keys and value pair
val outputRdd = moviesTupleRdd.join(ratingsTupleRdd)
outputRdd.take(3)

outputRdd: org.apache.spark.rdd.RDD[(Int, ((String, String), (Int, Double, Long)))] = MapPartitionsRDD[14] at join at <console>:29
res11: Array[(Int, ((String, String), (Int, Double, Long)))] = Array((140,((Up Close and Personal (1996),Drama|Romance),(1,3.0,942924980))), (140,((Up Close and Personal (1996),Drama|Romance),(2,3.5,1085569813))), (140,((Up Close and Personal (1996),Drama|Romance),(6,5.0,942843185))))
